In [157]:
import pandas as pd
import plotly.graph_objects as go

# Load Data

In [158]:
perhitungan = pd.read_csv('all_perhitungan.csv')

In [159]:
# drop unecessary columns
perhitungan.drop(['null'],axis=1,inplace=True)

In [160]:
caleg = pd.read_csv('data_caleg.csv')
caleg['id'] = caleg['id'].astype(str)
komeng = caleg[caleg['nama'].str.contains('KOMENG')].to_dict('records')[0]
komeng_id = str(komeng['id'])
komeng_nama = komeng['nama']

In [161]:
caleg_id = caleg['id'].astype(str).to_list()

caleg_except_komeng_id = caleg[~caleg['nama'].str.contains('KOMENG')]['id'].astype(str).to_list()


In [162]:
perhitungan_komeng = perhitungan[['kode_prov', 'kode_kab_kota', 'kode_kec', 'kode_desa', 'kode_tps',
       'prov', 'kab_kota', 'kecamatan', 'desa', 'tps', komeng_id, 'suara_sah', 'suara_total',
       'pemilih_dpt_j', 'pemilih_dpt_l', 'pemilih_dpt_p', 'pengguna_dpt_j',
       'pengguna_dpt_l', 'pengguna_dpt_p', 'pengguna_dptb_j',
       'pengguna_dptb_l', 'pengguna_dptb_p', 'suara_tidak_sah',
       'pengguna_total_j', 'pengguna_total_l', 'pengguna_total_p',
       'pengguna_non_dpt_j', 'pengguna_non_dpt_l', 'pengguna_non_dpt_p']]

In [163]:
suara_partai = pd.read_csv('detail_suara_partai.csv')
suara_presiden = pd.read_csv('detail_suara_presiden.csv')

# Group Perhitungan
- total tps
- total tps anomaly (https://x.com/A_Pangeran/status/1758336014911668587)
- total tps dalam proses (row with na value)
- total tps sah (selain anomaly dan memiliki value)

In [164]:
perhitungan['sum_suara_caleg'] = perhitungan[caleg_id].sum(axis=1)

In [165]:
perhitungan[['sum_suara_caleg','suara_total','suara_sah','suara_tidak_sah']]

,sum_suara_caleg,suara_total,suara_sah,suara_tidak_sah
0,0.0,NaN,NaN,NaN
1,213.0,235.0,213.0,22.0
2,137.0,160.0,136.0,24.0
3,181.0,239.0,201.0,38.0
4,429.0,218.0,189.0,29.0
...,...,...,...,...
140452,0.0,NaN,NaN,NaN
140453,140.0,192.0,145.0,47.0
140454,127.0,148.0,127.0,21.0
140455,139.0,159.0,139.0,20.0


### TPS Anomaly

In [166]:
#anomaly
anomaly1 = perhitungan[(perhitungan['suara_sah']==0) & (perhitungan['suara_total'] > 300)]
anomaly2 = perhitungan[(perhitungan['suara_sah']>0) & (perhitungan['suara_sah'] < perhitungan['sum_suara_caleg'])]
anomaly = pd.concat([anomaly1,anomaly2],axis=0,ignore_index=True)
len(anomaly)


7174

### TPS In Progress

In [167]:
in_progress = perhitungan[perhitungan.isnull().any(axis=1)]

### TPS Clean

In [168]:
anomaly_inprogress_index = anomaly.index.to_list() + in_progress.index.to_list()
clean = perhitungan.loc[~(perhitungan.index.isin(anomaly_inprogress_index))]

# chart
1. total uhuy (clean dan anomaly) 
2. total tidak uhuy (clean dan anomaly)
3. total suara (clean dan anomaly) 
6. total tps in progress
----
4. total tps win, lose distribution
5. total tps anomaly distribution
---
7. top 10 kota ter uhuy
8. top persentase uhuy
---
9. ~~distribusi uhuy~~
---
10. whos win on anomaly data
---
10. ~~anomaly~~
---
11. komparasi uhuy dengan partai
---
12. komparasi uhuy dengan pilpres




## Metrics

uhuy

In [169]:
# total uhuy clean
clean_uhuy = clean[komeng_id].sum()
# total uhuy anomaly
anomaly_uhuy = anomaly[komeng_id].sum()
# total uhuy
perhitungan_uhuy = perhitungan[komeng_id].sum()

print(clean_uhuy,anomaly_uhuy,perhitungan_uhuy)

1858365.0 276571.0 2263426.0


tidak uhuy

In [170]:
# total tidak uhuy clean
clean_tidak_uhuy = clean[caleg_except_komeng_id].sum(axis=1).sum()
# total tidak_uhuy anomaly
anomaly_tidak_uhuy = anomaly[caleg_except_komeng_id].sum(axis=1).sum()
# total tidak_uhuy
perhitungan_tidak_uhuy = perhitungan[caleg_except_komeng_id].sum(axis=1).sum()

print(clean_tidak_uhuy,anomaly_tidak_uhuy,perhitungan_tidak_uhuy)

8135154.0 2025178.0 9438606.0


total

In [171]:
# total clean
clean_total = clean[caleg_id].sum(axis=1).sum()
# total anomaly
anomaly_total = anomaly[caleg_id].sum(axis=1).sum()
# total 
perhitungan_total = perhitungan[caleg_id].sum(axis=1).sum()

print(clean_total,anomaly_total,perhitungan_total)

9993519.0 2301749.0 11702032.0


In [172]:
def human_format(num):
    num = float('{:.3g}'.format(num))
    magnitude = 0
    while abs(num) >= 1000:
        magnitude += 1
        num /= 1000.0
    return '{}{}'.format('{:f}'.format(num).rstrip('0').rstrip('.'), ['', 'K', 'M', 'B', 'T'][magnitude])


tps winning distribution

In [173]:
def is_komeng_win(row:pd.DataFrame):
    row[caleg_id].idxmax(axis=1)

In [174]:
clean['counter'] = 1
clean['winner'] = clean[caleg_id].idxmax(axis=1)

clean_komeng_win = clean[clean['winner']==komeng_id]
clean_komeng_lose = clean[clean['winner']!=komeng_id]

clean_komeng_win_count = clean_komeng_win.groupby('kab_kota')['counter'].sum().to_frame().reset_index().rename(columns={'counter':'win_count'})
clean_komeng_lose_count = clean_komeng_lose.groupby('kab_kota')['counter'].sum().to_frame().reset_index().rename(columns={'counter':'lose_count'})

clean_komeng_win_lose = pd.merge(clean_komeng_lose_count, clean_komeng_win_count,'inner','kab_kota')


C:\Users\yosia\AppData\Local\Temp\ipykernel_25764\401644962.py:1: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\yosia\AppData\Local\Temp\ipykernel_25764\401644962.py:2: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [175]:
clean_winning_percentage = (len(clean_komeng_win)/ (len(clean_komeng_win)+len(clean_komeng_lose))) * 100


In [176]:
# create custom html
def custom_indicator(title:str,value,total=None,ratio=None,anomaly=None):
    
    body = """<span style="font-size:1em;">{}</span>""".format(title)

    if total:
        body += """<br><span style="font-size:0.6em"> Total {} </span>""".format(total)
    if ratio:
        body += """<br><span style="font-size:0.6em"> {} </span>""".format(ratio)
    if anomaly:
        body += """<span style="font-size:0.6em;color:red">Anomaly {} </span>""".format(anomaly)
    
    body+="""<br><span style="font-size:2em;"><b>{}</b></span>""".format(value)
    body = f"""<div style="background-color:white; font-family: &quot;Open Sans&quot;, verdana, arial, sans-serif; fill-opacity: 1; white-space: pre;"><center>{body}</center></div>"""
    filename = 'html/custom_indicator_{}.html'.format(title.lower().replace('!','').replace(' ','_'))

    with open(filename,'w') as w:
        w.write(body)

    return body


#Winning TPS
custom_indicator("Winning TPS",value="{}%".format(round(clean_winning_percentage,2)),ratio="{}/{}".format(human_format(len(clean_komeng_win)),human_format((len(clean_komeng_win)+len(clean_komeng_lose)))))
#Suara Uhuy!
custom_indicator("Suara Uhuy!",value=human_format(clean_uhuy),total=human_format(perhitungan_uhuy),anomaly=human_format(anomaly_uhuy))
#Suara Tidak Uhuy!
custom_indicator("Suara Tidak Uhuy!",value=human_format(clean_tidak_uhuy),total=human_format(perhitungan_tidak_uhuy),anomaly=human_format(anomaly_tidak_uhuy))
#Suara Total
custom_indicator("Suara Total",value=human_format(clean_total),total=human_format(perhitungan_total),anomaly=human_format(anomaly_total))
   


'<div style="background-color:white; font-family: &quot;Open Sans&quot;, verdana, arial, sans-serif; fill-opacity: 1; white-space: pre;"><center><span style="font-size:1em;">Suara Total</span><br><span style="font-size:0.6em"> Total 11.7M </span><span style="font-size:0.6em;color:red">Anomaly 2.3M </span><br><span style="font-size:2em;"><b>9.99M</b></span></center></div>'

In [177]:
anomaly['counter'] = 1
anomaly['winner'] = anomaly[caleg_id].idxmax(axis=1)

anomaly_komeng_win = anomaly[anomaly['winner']==komeng_id]
anomaly_komeng_lose = anomaly[anomaly['winner']!=komeng_id]

anomaly_komeng_win_count = anomaly_komeng_win.groupby('kab_kota')['counter'].sum().to_frame().reset_index().rename(columns={'counter':'win_count'})
anomaly_komeng_lose_count = anomaly_komeng_lose.groupby('kab_kota')['counter'].sum().to_frame().reset_index().rename(columns={'counter':'lose_count'})

anomaly_komeng_win_lose = pd.merge(anomaly_komeng_lose_count, anomaly_komeng_win_count,'inner','kab_kota')


## winning

In [178]:
import plotly.express as px

fig = go.Figure()
fig.add_trace(go.Bar(name='Wining TPS',x=clean_komeng_win_lose['kab_kota'], y=clean_komeng_win_lose['win_count'],marker_color='#003E69',))
fig.add_trace(go.Bar(name='Losing TPS',x=clean_komeng_win_lose['kab_kota'], y=clean_komeng_win_lose['lose_count'],marker_color='#ED7D31'))
fig.update_layout(barmode='group', template='plotly_white', title={'text':'Wining By Kota Kabupaten'}, yaxis_title={'text':'Jumlah TPS'}, xaxis_title={'text':'Kota Kabupaten'})
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict( yanchor="top", y=0.9, xanchor="center", x=0.5
)
)
fig.update_layout(modebar_remove=[ 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut',])
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True
fig.write_html('html/bar_kota_kabupaten.html')
fig.show()

In [179]:

fig = go.Figure()
fig.add_trace(go.Bar(name='Wining TPS',x=anomaly_komeng_win_lose['kab_kota'], y=anomaly_komeng_win_lose['win_count'],marker_color='#003E69'))
fig.add_trace(go.Bar(name='Losing TPS',x=anomaly_komeng_win_lose['kab_kota'], y=anomaly_komeng_win_lose['lose_count'],marker_color='#ED7D31'))
fig.update_layout(barmode='group', template='plotly_white',title={'text':'Wining By Kota Kabupaten (Anomaly)'}, yaxis_title={'text':'Jumlah TPS'}, xaxis_title={'text':'Kota Kabupaten'})
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict( yanchor="top", y=0.9, xanchor="center", x=0.5)
)
fig.update_layout(modebar_remove=[ 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut',])
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True
fig.write_html('html/bar_kota_kabupaten_anomaly.html')
fig.show()

## kota ter uhuy

In [180]:
clean_sum = clean.groupby('kab_kota')[komeng_id].sum().to_frame().reset_index().rename(columns={komeng_id:'clean'})
anomaly_sum = anomaly.groupby('kab_kota')[komeng_id].sum().to_frame().reset_index().rename(columns={komeng_id:'anomaly'})
uhuy_by_kota = pd.merge(clean_sum,anomaly_sum,on='kab_kota',how='inner')
kota_ter_uhuy = uhuy_by_kota.sort_values('clean',ascending=False).head(10)

fig = go.Figure()
fig.add_trace(go.Bar(name='Suara Uhuy!',x=kota_ter_uhuy['kab_kota'], y=kota_ter_uhuy['clean'],marker_color='#003E69', hovertemplate="%{x}: %{y:.0f}"))
fig.add_trace(go.Bar(name='Anomaly',x=kota_ter_uhuy['kab_kota'], y=kota_ter_uhuy['anomaly'],marker_color='#ED7D31', hovertemplate="%{x}: %{y:.0f}"))
fig.update_layout(barmode='stack', template='plotly_white',title={'text':'10 Kota Kabupaten Ter-Uhuy!'}, yaxis_title={'text':'Jumlah Suara Uhuy!'}, xaxis_title={'text':'Kota Kabupaten'})
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict( yanchor="top", y=0.9, xanchor="center", x=0.5)

)
fig.update_layout(modebar_remove=[ 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut',])
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True
fig.write_html('html/bar_kota_teruhuy.html')
fig.show()


In [181]:
clean_sum_non_komeng = clean.groupby('kab_kota')[caleg_except_komeng_id].sum().sum(axis=1).to_frame().reset_index().rename(columns={0:'clean_non_uhuy'})
anomaly_sum_non_komeng = anomaly.groupby('kab_kota')[caleg_except_komeng_id].sum().sum(axis=1).to_frame().reset_index().rename(columns={0:'anomaly_non_uhuy'})
non_uhuy_by_kota = pd.merge(clean_sum_non_komeng,anomaly_sum_non_komeng,on='kab_kota',how='inner')

In [182]:
kepadatan_uhuy_by_kota = pd.merge(uhuy_by_kota,non_uhuy_by_kota,on='kab_kota',how='inner')
kepadatan_uhuy_by_kota['total_clean'] = kepadatan_uhuy_by_kota['clean']+kepadatan_uhuy_by_kota['clean_non_uhuy']
kepadatan_uhuy_by_kota['kepadatan'] = (kepadatan_uhuy_by_kota['clean']/kepadatan_uhuy_by_kota['total_clean'])
kepadatan_uhuy_by_kota['Persentase Kepadatan'] = kepadatan_uhuy_by_kota['kepadatan'].apply(lambda x:"{}%".format(round(x*100,2)))
uhuy_terpadat = kepadatan_uhuy_by_kota.sort_values('kepadatan',ascending=False).head(10)
uhuy_terpadat.rename(columns={'clean':'Memilih Komeng',"total_clean":"Total Suara"}, inplace=True)

# fig = go.Figure()
# fig.add_trace(go.Bar(name='Kepadatan Uhuy!',x=uhuy_terpadat['kab_kota'], y=uhuy_terpadat['kepadatan'],marker_color='#003E69',hovertemplate="%{y:%}"))

fig = px.bar(uhuy_terpadat,x='kab_kota',y='kepadatan', labels={'kab_kota':'Kota Kabupaten', 'kepadatan':'Kepadatan Suara Uhuy!'}, title='Konsentrasi Uhuy Kota Kabupaten Terpadat', hover_data={'kepadatan':False,'Persentase Kepadatan':True,"Memilih Komeng":True,"Total Suara":True})
fig.update_traces(marker_color='#003E69')
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    legend=dict( yanchor="top", y=0.9, xanchor="center", x=0.5)

)
fig.update_layout(modebar_remove=[ 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut',])
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True
fig.write_html('html/bar_konsentrasi_uhuy.html')
fig.show()


## Anomaly Winner

In [183]:
anomaly['winner'] = anomaly[caleg_id].idxmax(axis=1)

In [184]:
anomaly['winner_vote_sum'] = anomaly[caleg_id].max(axis=1).to_frame()

In [185]:
anomaly_winner = anomaly[['kode_prov','kode_kab_kota','kode_kec','kode_desa','kode_tps','kab_kota','kecamatan','desa','tps','winner','winner_vote_sum']].sort_values('winner_vote_sum',ascending=False).head(10)

In [186]:
anomaly_winner_caleg = anomaly_winner.merge(caleg[['id','nama']],how='left',left_on='winner',right_on='id')

In [187]:
def tps_url(row:pd.Series):
    # url = "https://pemilu2024.kpu.go.id/pemilu_dpd/hitung-suara/32/3201/320133/3201332003/3201332003010"
    url = "https://pemilu2024.kpu.go.id/pemilu_dpd/hitung-suara/{}/{}/{}/{}/{}".format(
        row['kode_prov'],
        row['kode_kab_kota'],
        row['kode_kec'],
        row['kode_desa'],
        row['kode_tps']
        )
    return url

In [188]:
def format_tps(row:pd.Series):
    alamat = """<span>{}, {}, {} <b> <a href="{}">{}</a> </b></span>""".format(row['kab_kota'],row['kecamatan'],row['desa'],row['url'],row['tps'])
    detail = "<span><b>{}</b></span><br>".format(row['nama'])+alamat
    return detail
    # return """<span><span> {} </span><span align="left" style="font-size:2em;"> {} Suara</span></span>""".format(detail,int(row['winner_vote_sum']))

In [189]:
anomaly_winner_caleg['url'] = anomaly_winner_caleg.apply(tps_url, axis=1)
anomaly_winner_caleg['alamat'] = anomaly_winner_caleg.apply(format_tps, axis=1)

In [190]:
anomaly_winner_caleg[['kab_kota','kecamatan','desa','tps','nama','winner_vote_sum','url']]

,kab_kota,kecamatan,desa,tps,nama,winner_vote_sum,url
0,BANDUNG,MAJALAYA,SUKAMAJU,TPS 035,MUHAMMAD MURTADLOILLAH,888.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
1,INDRAMAYU,LOHBENER,PAMAYAHAN,TPS 006,"HENDRIK KURNIAWAN, S.Pd.I.",888.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
2,BANDUNG,ARJASARI,RANCAKOLE,TPS 007,"AGITA NURFIANTI, S.Psi.",888.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
3,INDRAMAYU,LOHBENER,PAMAYAHAN,TPS 003,MULYADI ELHAN ZAKARIA,888.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
4,BANDUNG,ARJASARI,RANCAKOLE,TPS 008,"AMBU USDEK KANIAWATI, S.Sos.",888.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
5,INDRAMAYU,LOHBENER,PAMAYAHAN,TPS 005,"DENDA ALAMSYAH, S.T.",886.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
6,BANDUNG,ARJASARI,RANCAKOLE,TPS 029,"IMAM SOLAHUDIN, S.T., S.Ag., M.Si.",884.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
7,INDRAMAYU,LOHBENER,LANGUT,TPS 014,"ERNAWATY TAMPUBOLON, S.T., M.Th.",882.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
8,KARAWANG,JAYAKERTA,JAYAMAKMUR,TPS 017,"JAHENOS SARAGIH, S.Th., M.Th., M.M.",880.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...
9,BANDUNG,MARGAASIH,NANJUNG,TPS 020,Ir. ELAN HERYANTO,820.0,https://pemilu2024.kpu.go.id/pemilu_dpd/hitung...


In [191]:
fig = go.Figure(data=[go.Table(
    header=dict(values=['Detail','Jumlah Suara Pemenang']),
    columnwidth=[3,1],
    cells=dict(values=[
        anomaly_winner_caleg[column] for column in ['alamat','winner_vote_sum']
    ], align='left'))
                     ])
fig.update_layout(title={'text':'10 Pemenang di TPS Anomaly'})
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0)
)
fig.update_layout(modebar_remove=[ 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut',])
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True
fig.write_html('html/table_pemenang_tps_anomaly2.html')

fig.show()

In [192]:
#anomaly
perhitungan[(perhitungan['suara_sah']>0) & (perhitungan['suara_total'] > 300)][['kab_kota',	'kecamatan',	'desa',	'tps','sum_suara_caleg','suara_total','suara_sah','suara_tidak_sah']]

,kab_kota,kecamatan,desa,tps,sum_suara_caleg,suara_total,suara_sah,suara_tidak_sah
5427,BOGOR,GUNUNG SINDUR,RAWAKALONG,TPS 005,793.0,743.0,184.0,59.0
8568,BOGOR,RUMPIN,CIPINANG,TPS 013,130.0,343.0,239.0,104.0
11989,BOGOR,CIOMAS,PARAKAN,TPS 004,217.0,626.0,217.0,45.0
13401,BOGOR,CISEENG,CIBEUTEUNG UDIK,TPS 007,0.0,733.0,878.0,377.0
13543,BOGOR,CISEENG,CIBEUTEUNG MUARA,TPS 021,169.0,306.0,238.0,68.0
...,...,...,...,...,...,...,...,...
108712,BANDUNG BARAT,CILILIN,BONGAS,TPS 018,198.0,2019.0,199.0,20.0
136833,KOTA CIMAHI,CIMAHI SELATAN,LEUWIGAJAH,TPS 088,204.0,328.0,204.0,24.0
137676,KOTA CIMAHI,CIMAHI UTARA,CITEUREUP,TPS 082,222.0,1043.0,222.0,821.0
138159,KOTA TASIKMALAYA,CIPEDES,NAGARASARI,TPS 009,203.0,335.0,205.0,30.0


In [193]:
#anomaly
perhitungan[(perhitungan['suara_sah']>0) & (perhitungan['sum_suara_caleg'] > perhitungan['suara_sah'])][['sum_suara_caleg','suara_total','suara_sah','suara_tidak_sah']]

,sum_suara_caleg,suara_total,suara_sah,suara_tidak_sah
2,137.0,160.0,136.0,24.0
4,429.0,218.0,189.0,29.0
6,217.0,252.0,214.0,38.0
17,582.0,248.0,211.0,37.0
19,142.0,168.0,141.0,27.0
...,...,...,...,...
140352,128.0,159.0,120.0,39.0
140371,185.0,216.0,184.0,32.0
140394,181.0,206.0,176.0,30.0
140395,158.0,200.0,157.0,43.0


## comparison

In [194]:
suara_komeng = perhitungan[komeng_id].sum()
suara_komeng

2263426.0

In [195]:
suara_partai[['nama_lengkap','warna','jumlah_suara']].sort_values('jumlah_suara',ascending=False).head(5)

,nama_lengkap,warna,jumlah_suara
3,Partai Golongan Karya,#FFF051,1818326
1,Partai Gerakan Indonesia Raya,#990001,1700977
7,Partai Keadilan Sejahtera,#FC5100,1302843
0,Partai Kebangkitan Bangsa,#00764A,1256766
2,Partai Demokrasi Indonesia Perjuangan,#D52027,1221436


In [196]:
detail_chart_partai = [
    {"nickname":"PKB","nama_lengkap":"Partai Kebangkitan Bangsa","image_url":"https://upload.wikimedia.org/wikipedia/id/thumb/1/11/Logo_PKB.svg/160px-Logo_PKB.svg.png"},
    {"nickname":"Gerindra","nama_lengkap":"Partai Gerakan Indonesia Raya","image_url":"https://upload.wikimedia.org/wikipedia/id/thumb/9/9b/Lambang_Partai_Gerakan_Indonesia_Raya.svg/180px-Lambang_Partai_Gerakan_Indonesia_Raya.svg.png"},
    {"nickname":"Golkar","nama_lengkap":"Partai Golongan Karya","image_url":"//upload.wikimedia.org/wikipedia/commons/thumb/8/86/Logo_Golkar.svg/180px-Logo_Golkar.svg.png"},
    {"nickname":"PKS","nama_lengkap":"Partai Keadilan Sejahtera","image_url":"https://upload.wikimedia.org/wikipedia/commons/thumb/d/d2/Partai_Keadilan_Sejahtera_2020.svg/150px-Partai_Keadilan_Sejahtera_2020.svg.png"},
    {"nickname":"PDIP","nama_lengkap":"Partai Demokrasi Indonesia Perjuangan","image_url":"//upload.wikimedia.org/wikipedia/id/thumb/d/d8/Lambang_Partai_Demokrasi_Indonesia_Perjuangan.svg/180px-Lambang_Partai_Demokrasi_Indonesia_Perjuangan.svg.png"},
]
detail_partai = pd.DataFrame(detail_chart_partai)
suara_partai_detailed = suara_partai.merge(detail_partai,how='inner',on='nama_lengkap')
suara_partai_detailed.sort_values('jumlah_suara',ascending=False,inplace=True)
    # "Komeng":"https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/KPU_Alfiansyah_Bustami_Komeng.jpg/1200px-KPU_Alfiansyah_Bustami_Komeng.jpg" #397ae3

In [197]:
# add suara komeng
suara_komeng_df = pd.DataFrame([{'nama':'Komeng','nickname':'Komeng','warna':'#397ae3','jumlah_suara':suara_komeng,'image_url':'https://upload.wikimedia.org/wikipedia/commons/thumb/2/25/KPU_Alfiansyah_Bustami_Komeng.jpg/1200px-KPU_Alfiansyah_Bustami_Komeng.jpg',}])
suara_komeng_df

,nama,nickname,warna,jumlah_suara,image_url
0,Komeng,Komeng,#397ae3,2263426.0,https://upload.wikimedia.org/wikipedia/commons...


In [198]:
suara_komeng_vs_partai = pd.concat([suara_komeng_df,suara_partai_detailed])
suara_komeng_vs_partai

,nama,nickname,warna,jumlah_suara,image_url,ts,id_partai,id_pilihan,is_aceh,nama_lengkap,nomor_urut
0,Komeng,Komeng,#397ae3,2263426.0,https://upload.wikimedia.org/wikipedia/commons...,NaN,NaN,NaN,NaN,NaN,NaN
3,Partai Golongan Karya,Golkar,#FFF051,1818326.0,//upload.wikimedia.org/wikipedia/commons/thumb...,2024-02-17 16:00:02,100004.0,4.0,False,Partai Golongan Karya,4.0
1,Partai Gerakan Indonesia Raya,Gerindra,#990001,1700977.0,https://upload.wikimedia.org/wikipedia/id/thum...,2024-02-17 16:00:02,100002.0,2.0,False,Partai Gerakan Indonesia Raya,2.0
4,Partai Keadilan Sejahtera,PKS,#FC5100,1302843.0,https://upload.wikimedia.org/wikipedia/commons...,2024-02-17 16:00:02,100008.0,8.0,False,Partai Keadilan Sejahtera,8.0
0,Partai Kebangkitan Bangsa,PKB,#00764A,1256766.0,https://upload.wikimedia.org/wikipedia/id/thum...,2024-02-17 16:00:02,100001.0,1.0,False,Partai Kebangkitan Bangsa,1.0
2,Partai Demokrasi Indonesia Perjuangan,PDIP,#D52027,1221436.0,//upload.wikimedia.org/wikipedia/id/thumb/d/d8...,2024-02-17 16:00:02,100003.0,3.0,False,Partai Demokrasi Indonesia Perjuangan,3.0


In [199]:
#color 
bar_color = {}
for _,row in suara_komeng_vs_partai.iterrows():
    bar_color[row['nickname']] = row['warna']

# Step 3: Create Plotly bar chart
# fig = go.Figure(data=[go.Bar(x=suara_komeng_vs_partai['nama'], y=suara_komeng_vs_partai['jumlah_suara'],color_discrete_map = bar_color)])
fig = px.bar(suara_komeng_vs_partai, x='nickname', y='jumlah_suara',color='nickname',color_discrete_map=bar_color)
fig.update_traces(hovertemplate="%{x}: %{y:.0f}")
# fig = go.Figure()

# Step 4: Add annotations with images to each bar
counter = 0.5
for _,row in suara_komeng_vs_partai.iterrows():
    fig.add_layout_image(
        dict(
            source=row['image_url'],
            xref="paper",
            yref="paper",
            x=counter/len(suara_komeng_vs_partai),
            y=0.25,
            sizex=0.2,
            sizey=0.2,
            xanchor="center",
            yanchor="middle",
            sizing="contain",
            opacity=1,
            
            # layer="below"
        )
    )
    counter+=1

fig.update_layout( 
    xaxis=dict(title='Nama'),
    yaxis=dict(title='Jumlah Suara'),
    title='Suara Uhuy VS Suara Pileg DPR (Jawa Barat)'
)
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    showlegend=False
)
fig.update_layout(modebar_remove=[ 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut',])
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True
fig.write_html('html/bar_komeng_vs_pileg_dpr.html')
fig.show()


In [200]:
#suara presiden
suara_presiden

,ts,nama,warna,nomor_urut,id,jumlah_suara
0,2024-02-17 16:00:02,"H. ANIES RASYID BASWEDAN, Ph.D. - Dr. (H.C.) H...",#8CB9BD,1,100025,6490745
1,2024-02-17 16:00:02,H. PRABOWO SUBIANTO - GIBRAN RAKABUMING RAKA,#C7B7A3,2,100026,12009854
2,2024-02-17 16:00:02,"H. GANJAR PRANOWO, S.H., M.I.P. - Prof. Dr. H....",#B67352,3,100027,2026611


In [201]:
detail_chart_presiden = [
    {"nickname":"Anies - Muhaimin", "warna":"#a5d63c","nama":"H. ANIES RASYID BASWEDAN, Ph.D. - Dr. (H.C.) H. A. MUHAIMIN ISKANDAR","image_url":"https://asset.kompas.com/data/2024/02/19/kompascom/widget/realcount/images/widget/real-count/paslon-1.png"},
    {"nickname":"Prabowo - Gibran","warna":"#09bce3","nama":"H. PRABOWO SUBIANTO - GIBRAN RAKABUMING RAKA","image_url":"https://asset.kompas.com/data/2024/02/19/kompascom/widget/realcount/images/widget/real-count/paslon-2.png"},
    {"nickname":"Ganjar - Mahfud","warna":"#e65225","nama":"H. GANJAR PRANOWO, S.H., M.I.P. - Prof. Dr. H. M. MAHFUD MD","image_url":"https://asset.kompas.com/data/2024/02/19/kompascom/widget/realcount/images/widget/real-count/paslon-3.png"},
]
detail_presiden = pd.DataFrame(detail_chart_presiden)
suara_presiden_detailed = suara_presiden[['nama','jumlah_suara']].merge(detail_presiden,how='inner',on='nama')
# suara_presiden_detailed.sort_values('jumlah_suara',ascending=False,inplace=True)

In [202]:
suara_komeng_vs_presiden = pd.concat([suara_presiden_detailed,suara_komeng_df])
# suara_komeng_vs_presiden.sort_values('jumlah_suara',ascending=False,inplace=True)
suara_komeng_vs_presiden

,nama,jumlah_suara,nickname,warna,image_url
0,"H. ANIES RASYID BASWEDAN, Ph.D. - Dr. (H.C.) H...",6490745.0,Anies - Muhaimin,#a5d63c,https://asset.kompas.com/data/2024/02/19/kompa...
1,H. PRABOWO SUBIANTO - GIBRAN RAKABUMING RAKA,12009854.0,Prabowo - Gibran,#09bce3,https://asset.kompas.com/data/2024/02/19/kompa...
2,"H. GANJAR PRANOWO, S.H., M.I.P. - Prof. Dr. H....",2026611.0,Ganjar - Mahfud,#e65225,https://asset.kompas.com/data/2024/02/19/kompa...
0,Komeng,2263426.0,Komeng,#397ae3,https://upload.wikimedia.org/wikipedia/commons...


In [203]:
#color 
bar_color = {}
for _,row in suara_komeng_vs_presiden.iterrows():
    bar_color[row['nickname']] = row['warna']

# Step 3: Create Plotly bar chart
# fig = go.Figure(data=[go.Bar(x=suara_komeng_vs_presiden['nama'], y=suara_komeng_vs_presiden['jumlah_suara'],color_discrete_map = bar_color)])
fig = px.bar(suara_komeng_vs_presiden, x='nickname', y='jumlah_suara',color='nickname', color_discrete_map=bar_color)
fig.update_traces(hovertemplate="%{x}: %{y:.0f}")

# fig = go.Figure()

# Step 4: Add annotations with images to each bar
counter = 0.5
for _,row in suara_komeng_vs_presiden.iterrows():
    fig.add_layout_image(
        dict(
            source=row['image_url'],
            xref="paper",
            yref="paper",
            x=counter/len(suara_komeng_vs_presiden),
            y=0.25,
            sizex=0.3,
            sizey=0.3,
            xanchor="center",
            yanchor="middle",
            sizing="contain",
            opacity=1,
            
            # layer="below"
        )
    )
    counter+=1

fig.update_layout(
    xaxis=dict(title='Nama'),
    yaxis=dict(title='Jumlah Suara'),
    title='Suara Uhuy VS Suara Pemilihan Presiden (Jawa Barat)'
)
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    showlegend=False
)
fig.update_layout(modebar_remove=[ 'zoom', 'pan', 'select', 'lasso2d', 'zoomIn', 'zoomOut',])
fig.layout.xaxis.fixedrange = True
fig.layout.yaxis.fixedrange = True
fig.write_html('html/bar_komeng_vs_pilpres.html')
fig.show()


In [204]:
suara_komeng_vs_presiden

,nama,jumlah_suara,nickname,warna,image_url
0,"H. ANIES RASYID BASWEDAN, Ph.D. - Dr. (H.C.) H...",6490745.0,Anies - Muhaimin,#a5d63c,https://asset.kompas.com/data/2024/02/19/kompa...
1,H. PRABOWO SUBIANTO - GIBRAN RAKABUMING RAKA,12009854.0,Prabowo - Gibran,#09bce3,https://asset.kompas.com/data/2024/02/19/kompa...
2,"H. GANJAR PRANOWO, S.H., M.I.P. - Prof. Dr. H....",2026611.0,Ganjar - Mahfud,#e65225,https://asset.kompas.com/data/2024/02/19/kompa...
0,Komeng,2263426.0,Komeng,#397ae3,https://upload.wikimedia.org/wikipedia/commons...


In [205]:
bar_color

{'Anies - Muhaimin': '#a5d63c',
 'Prabowo - Gibran': '#09bce3',
 'Ganjar - Mahfud': '#e65225',
 'Komeng': '#397ae3'}